### Imports

In [1]:
## Python Files
import utils
import datas
import models

## Libraries
import os  # when loading file paths

from PIL import Image  # Load img
import torch
import torch.nn as nn
import statistics
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import mat73

### Setting up GPU Device

In [2]:
# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True

Using device: cpu



### Convert .mat to .png images for memory managment during training

In [3]:
path2dataset = "nyu_depth_v2_labeled.mat"
# utils.mat2png(path2dataset)

### Load datas

In [3]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import datas

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


root_folder = "Datas"

transform = get_transform(train=True)

train_loader, test_loader = datas.get_loader(root_folder, batch_size=2, num_datas=1449, train_test_ratio=0.9, transform=transform)

### Training

In [5]:
learning_rate = 0.01
num_classes = 894
object_detector = models.ObjectDetector(num_classes=num_classes).to(device)
criterion = nn.MSELoss().to(device)
params = [p for p in object_detector.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=learning_rate,
                            momentum=0.9, weight_decay=0.0005)

# learning rate scheduler which decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [10]:
from torch.optim.lr_scheduler import StepLR
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(object_detector, optimizer, train_loader, device, epoch, print_freq=1)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(object_detector, data_loader_test, device=device)

/Users/mohammadaminalamalhod/anaconda3/envs/DL/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: [0]  [  0/652]  eta: 9:47:34  lr: 0.000000  loss: 7.3955 (7.3955)  loss_classifier: 5.6536 (5.6536)  loss_box_reg: 0.8259 (0.8259)  loss_mask: 0.6950 (0.6950)  loss_objectness: 0.1063 (0.1063)  loss_rpn_box_reg: 0.1147 (0.1147)  time: 54.0710  data: 0.1663


KeyboardInterrupt: 

In [12]:
# pick one image from the test set
images, targets = next(iter(train_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
img = images[0]
# put the model in evaluation mode
object_detector.eval()
with torch.no_grad():
    prediction = object_detector([img.to(device)])

torch.Size([3, 480, 640])


In [13]:
prediction

[{'boxes': tensor([], size=(0, 4)),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([]),
  'masks': tensor([], size=(0, 1, 480, 640))}]